## CRAGIG Runs 1 and 2 through ipyrad

**20170605**

I'm going to run all my current scallop RAD data through the ipyrad pipeline because as of yet I have only run it on ten samples at a time to get the hang of it and to see preliminary differences between Stacks and ipyrad.

First, I need to demultiplex my two lanes of data. For each, I'll make a unique params file for each library and then I'll merge them together.

The two params files will differ only in the location of the raw data fastq file and the barcodes files.

cragig007 will be for CRAGIG_run1 and cragig008 will be for CRAGIG_run2

This is what's in cragig007:

In [1]:
cd /mnt/hgfs/SHARED_FOLDER/Learn_iPyrad/CRAGIG_RUNS_12_py

/mnt/hgfs/SHARED_FOLDER/Learn_iPyrad/CRAGIG_RUNS_12_py


In [8]:
ls

barcodes_cragigrun1.txt*  params-cragig007.txt*
barcodes_cragigrun2.txt*  popmap_cragig007.txt*


In [11]:
!head -n 30 params-cragig007.txt

------- ipyrad params file (v.0.6.20)-------------------------------------------
cragig007                      ## [0] [assembly_name]: Assembly name. Used to name output directories for assembly steps
./                             ## [1] [project_dir]: Project dir (made in curdir if not present)
../../../Data/CRAGIG_RUN1_rawdata/161228_I137_FCHCYV5BBXX_L5_CHKPE85216120009_2.fq.gz                               ## [2] [raw_fastq_path]: Location of raw non-demultiplexed fastq files
./barcodes_cragigrun1.txt     ## [3] [barcodes_path]: Location of barcodes file
                              ## [4] [sorted_fastq_path]: Location of demultiplexed/sorted fastq files
denovo                         ## [5] [assembly_method]: Assembly method (denovo, reference, denovo+reference, denovo-reference)
                               ## [6] [reference_sequence]: Location of reference sequence file
rad                            ## [7] [datatype]: Datatype (see docs): rad, gbs, ddrad, etc.
TGCAGG,      

In [17]:
!ipyrad -p params-cragig007.txt -s 1


 -------------------------------------------------------------
  ipyrad [v.0.6.20]
  Interactive assembly and analysis of RAD-seq data
 -------------------------------------------------------------

  A new version of ipyrad is available (v.0.6.27). To upgrade run:

    conda install -c ipyrad ipyrad


 -------------------------------------------------------------
  ipyrad [v.0.6.20]
  Interactive assembly and analysis of RAD-seq data
 -------------------------------------------------------------
  New Assembly: cragig007
  host compute node: [6 cores] on ubuntu

  Step 1: Demultiplexing fastq data to Samples
  [####################] 100%  sorting reads         | 1:26:47  es  | 0:25:59  
  [####################] 100%  writing/compressing   | 0:37:57  



In [ ]:
!ipyrad -p params-cragig008.txt -s 1


 -------------------------------------------------------------
  ipyrad [v.0.6.20]
  Interactive assembly and analysis of RAD-seq data
 -------------------------------------------------------------

  A new version of ipyrad is available (v.0.6.27). To upgrade run:

    conda install -c ipyrad ipyrad


 -------------------------------------------------------------
  ipyrad [v.0.6.20]
  Interactive assembly and analysis of RAD-seq data
 -------------------------------------------------------------
  New Assembly: cragig008
  host compute node: [6 cores] on ubuntu

  Step 1: Demultiplexing fastq data to Samples
  [##################  ]  92%  sorting reads         | 0:39:39  

It kept crashing the jupyter notebook so I'm starting to run these outside the terminal. For some reason, even though I merged assemblies, it only ran on the first 71 samples. Here's what the params file looked like:

In [2]:
!head -n 30 params-cragig009.txt

------- ipyrad params file (v.0.6.20)-------------------------------------------
cragig009                      ## [0] [assembly_name]: Assembly name. Used to name output directories for assembly steps
/mnt/hgfs/SHARED_FOLDER/Learn_iPyrad/CRAGIG_RUNS_12_py ## [1] [project_dir]: Project dir (made in curdir if not present)
Merged: cragig007, cragig008   ## [2] [raw_fastq_path]: Location of raw non-demultiplexed fastq files
Merged: cragig007, cragig008   ## [3] [barcodes_path]: Location of barcodes file
Merged: cragig007, cragig008   ## [4] [sorted_fastq_path]: Location of demultiplexed/sorted fastq files
denovo                         ## [5] [assembly_method]: Assembly method (denovo, reference, denovo+reference, denovo-reference)
                               ## [6] [reference_sequence]: Location of reference sequence file
rad                            ## [7] [datatype]: Datatype (see docs): rad, gbs, ddrad, etc.
TGCAGG,                        ## [8] [restriction_overhang]: Restrictio

Aw bummer. It looks like the fastsq files for both cragig007 and cragig0008 are the same, so I might have goofed in what files I fed it. Double-checking the params files now... Found it! I didn't use the correct barcodes file (I used the one for CRAGIG_run1 for both cragig007 and cragig008). Going to double-check the params files and rerun.

Watching it rerun and ipyrad knows to delete all old files when I force the step to rewrite over the previous version. That's pretty cool!

I'm not totally sure that I know how to use the merge operation well yet, as it spit out an error during Step 2:
```
ERROR 	IPyradError(error in: ['cat', '/mnt/hgfs/SHARED_FOLDER/Learn_iPyrad/CRAGIG_RUNS_12_py/cragig007_fastqs/FG034_R1_.fastq.gz', '/mnt/hgfs/SHARED_FOLDER/Learn_iPyrad/CRAGIG_RUNS_12_py/cragig008_fastqs/FG034_R1_.fastq.gz'], None)
```

I'm going to rerun evertyhing from the start, and double-check I have the merge function well understood.